In [1]:
import os
import subprocess as sp
from pathlib import Path
import itertools
from osgeo import gdal, ogr
import numpy as np
from datetime import datetime

In [2]:
def cmd(command, parse_directly=False):
    
    if parse_directly==True:
        #print(command)
        norm = sp.Popen(command,stdout=sp.PIPE, shell=True)
    else:
        #print(sp.list2cmdline(command))
        norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)            
             
    out_cmd = norm.communicate()
    return out_cmd

In [3]:
def get_WD_folder(base, workdir):
    base_list = base.split('_')[1]
    if 'A' in base_list:
        coef = 'ruweData_A'        

    if 'B' in base_list:
        coef = 'ruweData_B'
    
    wd_dir = 'WD_{}_{}'.format(base_list[6].zfill(3), 
                                   base_list[7].zfill(3))    
    
    workdir_WD = os.path.join(workdir, coef, wd_dir)   
    return workdir_WD

In [4]:
def iter_incrementing_file_names(path):
    """
    Iterate incrementing file names. Start with path and add " (n)" before the
    extension, where n starts at 1 and increases.

    :param path: Some path
    :return: An iterator.
    """
    yield path
    prefix, ext = os.path.splitext(path)
    for i in itertools.count(start=1, step=1):
        no = str(i).zfill(2)
        # yield prefix + '_{0}'.format(i) + ext
        yield prefix + '_' + no + ext

In [5]:
def get_unique_filename(file_in):
    for filename in iter_incrementing_file_names(file_in):
        new_fn = Path(filename)
        if new_fn.is_file():
            pass
        else:
            return filename

In [6]:
def get_features(shape):
    driver = ogr.GetDriverByName('ESRI Shapefile')

    dataSource = driver.Open(shape, 0)  # 0 means read-only. 1 means writeable.

    # Check to see if shapefile is found.
    if dataSource is None:
        print('Could not open %s' % (shape))
        return
    else:
        print('Opened %s' % (shape))
        layer = dataSource.GetLayer()
        shape_features = layer.GetFeatureCount()
        print('Name of layer: %s' % layer.GetDescription())
        print("Number of features in %s: %d" % (os.path.basename(shape), shape_features))

        features_shape = []
        for i in range(shape_features):
            feat = layer.GetFeature(i)            
            obID = feat.GetField('OBJECTID')
            
            geom = feat.GetGeometryRef()
            extent = geom.GetEnvelope()
            #print(extent)
            
            features_shape.append({'obID':obID, 'extent':list(set(extent))})
        #features_shape = list(set(features_shape))
        return dataSource, layer, features_shape

In [7]:
rootDir = r'D:\FEWSProjecten\OmsWaddenzee\trunk\fews'
configDir = r'D:\FEWSProjecten\OmsWaddenzee\trunk\fews\Config'

sourceRoot = os.path.join(rootDir, r'Import\hkv\coefficienten\bronShp version 2.1')
tmpRoot = os.path.join(rootDir, r'ImportInterim\hkv\coefficienten\tmpData')
backupRoot = os.path.join(rootDir, r'ImportBackup\hkv\coefficienten\ruweDataBackup')
asciiRoot = os.path.join(rootDir, r'Import\hkv\coefficienten\asciiData')

y = os.path.join(configDir, r'MapLayerFiles\Achtergrond_polygonen//Achtergrond_polygonen.shp')

In [8]:
rootOgrGdal = r'C:\Python35\Lib\site-packages\osgeo'
python_exe = r'C:\Python35\python.exe'
ogr2ogr = os.path.join(rootOgrGdal, 'ogr2ogr.exe')
gdalwarp = os.path.join(rootOgrGdal, 'gdalwarp.exe')
gdal_fillnodata = os.path.join(rootOgrGdal, 'scripts','gdal_fillnodata.py')
gdal_rasterize = os.path.join(rootOgrGdal, 'gdal_rasterize.exe')
gdal_translate = os.path.join(rootOgrGdal, 'gdal_translate.exe')
gdalbuildvrt = os.path.join(rootOgrGdal, 'gdalbuildvrt.exe')
gdalinfo = os.path.join(rootOgrGdal, 'gdalinfo.exe')
ogrinfo = os.path.join(rootOgrGdal, 'ogrinfo.exe')

In [9]:
# get id and extents of all features in achtergrond_polygonen shapefile
try:
    ds, layer, z_features = get_features(y)
    l = layer.GetDescription()    
except Exception as ex:
    print(ex)
    # continue

Opened D:\FEWSProjecten\OmsWaddenzee\trunk\fews\Config\MapLayerFiles\Achtergrond_polygonen//Achtergrond_polygonen.shp
Name of layer: Achtergrond_polygonen
Number of features in Achtergrond_polygonen.shp: 481


In [10]:
#today date
dt = datetime.today().strftime('%Y%m%d')+'000000'

In [11]:
for root, dirs, files in os.walk(sourceRoot):
    for file in files:
        if file.endswith('.shp') and 'test' not in file:            
            
            init_file = os.path.join(os.path.abspath(root), file)
            base, extension = os.path.splitext(init_file)
            #folders            
            coef = base.split('\\')[-3]
            wd = base.split('\\')[-2]
            fl = base.split('\\')[-1]
            
            
            
            sourcedir = os.path.join(sourceRoot, coef, wd)
            workdir = os.path.join(tmpRoot, coef, wd)
            backupdir = os.path.join(backupRoot, coef, wd)
            asciiDataDir = os.path.join(asciiRoot, coef, wd)
            
            tile = fl.split('_')[1]
            
            # get feature 
            obID = int(tile) - 1
            feature = z_features[obID]
            x = [str(e) for e in np.sort(np.round(feature['extent'], 1))]
            #print(x)
            t = tile.zfill(3)
            a = init_file
            
            # 0 convert point SHP to RASTER
            b = os.path.join(workdir, 'step0_{}.tif'.format(t))
            command = [gdal_rasterize, '-a', 'Z', '-a_nodata', '-9999', '-a_srs', 'EPSG:28992',
                       '-tr', '1.0', '1.0', '-te', x[0], x[2], x[1], x[3], '-l', fl, a, b]
            norm = cmd(command)
            #print(norm[0])

            # 1 since grids contain scattered data, interpolate and fill the nodata values to 100m edge 
            c = os.path.join(workdir, 'step1_{}.tif'.format(t))
            command = [python_exe, gdal_fillnodata, '-q', '-md', '100', b, c]
            norm = cmd(command)
            #print(norm[0])

            # 2 merge tif to ascii-grid
            f_out = 'grid{}_{}.asc'.format(t, dt)
            d = os.path.join(asciiDataDir, f_out)
            command = [gdal_translate, '-of', 'AAIGrid', '-a_nodata', '-9999', c, d]
            norm = cmd(command)               
            #print(norm[0])
            print(coef, wd, fl, f_out)

ruweData_A WD_000_090 clip_1 grid001_20190218000000.asc
ruweData_A WD_000_090 clip_10 grid010_20190218000000.asc
ruweData_A WD_000_090 clip_100 grid100_20190218000000.asc
ruweData_A WD_000_090 clip_101 grid101_20190218000000.asc
ruweData_A WD_000_090 clip_102 grid102_20190218000000.asc
ruweData_A WD_000_090 clip_103 grid103_20190218000000.asc
ruweData_A WD_000_090 clip_104 grid104_20190218000000.asc
ruweData_A WD_000_090 clip_105 grid105_20190218000000.asc
ruweData_A WD_000_090 clip_106 grid106_20190218000000.asc
ruweData_A WD_000_090 clip_107 grid107_20190218000000.asc
ruweData_A WD_000_090 clip_108 grid108_20190218000000.asc
ruweData_A WD_000_090 clip_109 grid109_20190218000000.asc
ruweData_A WD_000_090 clip_11 grid011_20190218000000.asc
ruweData_A WD_000_090 clip_110 grid110_20190218000000.asc
ruweData_A WD_000_090 clip_111 grid111_20190218000000.asc
ruweData_A WD_000_090 clip_112 grid112_20190218000000.asc
ruweData_A WD_000_090 clip_113 grid113_20190218000000.asc
ruweData_A WD_000_

In [12]:
# purge redundant xml and prj files 
xmlFiles = []
prjFiles = []
for root, dirs, files in os.walk(asciiRoot):
    for file in files:
        if file.endswith(".xml"):
             xmlFiles.append(os.path.join(root, file))
        if file.endswith(".prj"):
             prjFiles.append(os.path.join(root, file))               

for xmlFile in xmlFiles:
    os.remove(xmlFile)

for prjFile in prjFiles:
    os.remove(prjFile)   

In [ ]:
asciiRoot